## Задание

Реализовать обучение линейной регрессии для задачи boston house prices 
(https://www.kaggle.com/vikrishnan/boston-house-prices) с использованием torch’а

In [1]:
import torch
from torch import autograd
from sklearn.datasets import load_boston

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import time

In [2]:
columns = ['CRIM', 'ZN', 'INDUS', 'CHAS','NOX','RM', 'AGE','DIS', 'RAD','TAX', 'PTRATIO','B', 'LSTAT', 'MEDV'] 

In [3]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

In [4]:
# Преобразование данных 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
num_inputs = X.shape[1]
num_examples = X.shape[0]
w = torch.randn(X.shape[1]) # todo - xavie initializing
b = torch.rand(1)# todo - xavie initializing
features_train, features_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
labels_train, labels_test = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32) 

In [6]:
# Чтение данных

In [7]:
def data_iter(batch_size, features, labels):
    num_examples = len(features) # число 100
    indices = list(range(num_examples)) # список чисел от 0 до 99
    # The examples are read at random, in no particular order
    random.shuffle(indices) # перемешиваем этот список
    for i in range(0, num_examples, batch_size): 
        j = indices[i: min(i + batch_size, num_examples)] # Выбираем индексы для батчка
        yield features[j, :], labels[j] # возвращаем батч

In [8]:
# Тест маленького батча

In [9]:
batch_size = 10
for X, y in data_iter(batch_size, features_train, labels_train):
    print(X, y)
    break

tensor([[-0.5512,  4.3177, -1.3602, -0.2513, -1.2520,  2.6566, -1.4106,  0.7376,
         -0.6510, -1.1328, -1.8325,  0.4114, -1.4210],
        [ 1.6868, -0.4778,  1.0755, -0.2513,  1.5400,  0.4286,  0.7771, -0.8795,
          1.7709,  1.6481,  0.7361, -3.4216,  0.6265],
        [-0.5223,  0.6327, -0.8579, -0.2513, -1.1236, -0.8009,  0.1785,  2.2102,
         -0.2877, -0.4659,  0.2224,  0.2208, -0.1226],
        [-0.3438, -0.4778, -0.4977, -0.2513, -0.2013, -0.1224,  1.0159,  0.2481,
         -0.6510, -0.6106,  1.1098,  0.2655, -0.0497],
        [-0.4296, -0.4778, -0.4977, -0.2513, -0.2013, -0.9949, -1.2431,  0.0517,
         -0.6510, -0.6106,  1.1098, -0.6608, -0.2319],
        [-0.5427,  1.1880, -1.4392, -0.2513, -0.7702,  1.7869,  0.0146, -0.3103,
         -0.2877, -1.1454, -0.1045,  0.4785, -0.9365],
        [-0.4026, -0.4778,  1.6742, -0.2513,  0.5401, -0.5686,  0.9589, -0.7013,
         -0.6510,  0.2073,  1.2032, -0.9377,  0.5266],
        [ 0.4471, -0.4778,  1.0755, -0.2513,  0.

In [10]:
model = torch.nn.Sequential(torch.nn.Linear(13, 1))
loss = torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')
trainer = torch.optim.SGD(model.parameters(), lr=0.001)

In [11]:
num_epochs = 300
train_losses = []
test_losses = []

for epoch in range(0, num_epochs):
    start, train_loss, train_mse, train_n, test_loss, test_mse, test_n = time.time(), 0., 0., 0, 0., 0., 0
    
    # режим обучения
    model.train()
    for X, y in data_iter(batch_size, features_train, labels_train):
        trainer.zero_grad() # не забывать обнуление градента!
        y_hat = model.forward(X)
        l = loss(y_hat, y)
        l.backward()
        trainer.step()
        train_losses.append(l.item())
        train_loss += l.item()
        train_n += len(X)
    
    # режим вычисления
    model.eval()
    for X, y in data_iter(batch_size, features_test, labels_test):
        y_hat = model.forward(X)
        l = loss(y_hat, y)
        test_losses.append(l.item())
        test_loss += l.item()
        test_n += len(X)   
    

    print(f'epoch {epoch}, taked: {round(time.time() - start,5)},' 
          + f'train -> loss: {train_loss / train_n}, test -> loss: {round(test_loss / test_n, 5)},'
          #+ f'w : {model[0].weight.data}, b :{model[0].bias.data}'
         )

C:\Users\mi\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\mi\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\mi\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, tar

epoch 0, taked: 0.02345,train -> loss: 59.1282321438931, test -> loss: 59.09167,
epoch 1, taked: 0.01699,train -> loss: 54.05265959182588, test -> loss: 57.06777,
epoch 2, taked: 0.01197,train -> loss: 55.00312971360613, test -> loss: 54.85545,
epoch 3, taked: 0.01296,train -> loss: 70.18394455107132, test -> loss: 52.9015,
epoch 4, taked: 0.01135,train -> loss: 50.275262964834084, test -> loss: 51.48252,
epoch 5, taked: 0.01197,train -> loss: 47.79214024307704, test -> loss: 49.12699,
epoch 6, taked: 0.012,train -> loss: 47.39562791880995, test -> loss: 47.49035,
epoch 7, taked: 0.012,train -> loss: 45.21278819471303, test -> loss: 46.06177,
epoch 8, taked: 0.01093,train -> loss: 44.880788973062344, test -> loss: 45.46998,
epoch 9, taked: 0.01196,train -> loss: 43.563815579556, test -> loss: 42.93736,
epoch 10, taked: 0.01101,train -> loss: 43.640701596099554, test -> loss: 41.27314,
epoch 11, taked: 0.01194,train -> loss: 39.4257113012937, test -> loss: 39.98873,
epoch 12, taked: 0.0

In [12]:
columns = ['CRIM', 'ZN', 'INDUS', 'CHAS','NOX','RM', 'AGE','DIS', 'RAD','TAX', 'PTRATIO','B', 'LSTAT', 'MEDV']  
pd.Series(model[0].weight.data.numpy().flatten(), index=columns[:-1]).sort_values(ascending=False)

RM         0.409806
RAD        0.381300
B          0.250312
ZN         0.195078
TAX        0.129182
CHAS       0.115291
AGE        0.094818
INDUS      0.093596
CRIM      -0.231026
PTRATIO   -0.373536
NOX       -0.529726
DIS       -0.529819
LSTAT     -0.886847
dtype: float32

In [13]:
model[0].bias.data.numpy()

array([21.450083], dtype=float32)

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)
pd.Series(lr.coef_, index=columns[:-1]).sort_values(ascending=False)

RAD        3.559998
RM         1.691747
AGE        1.332263
CHAS       0.954143
B          0.873937
INDUS      0.730919
ZN         0.722100
CRIM      -0.811797
TAX       -1.681608
PTRATIO   -2.762897
DIS       -3.614028
NOX       -4.183770
LSTAT     -5.461820
dtype: float64

In [15]:
lr.intercept_

21.502970297029673